In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, TimeDistributed, GRU, LSTM
from tensorflow.keras.models import Model
import tensorflow as tf

In [2]:
import numpy as np
import string
import csv

In [3]:
# max word length
maxlen = 21
# LSTM time step (step letters each pass)
step = 1

# train the model directly
TRAIN = False

In [4]:
if TRAIN == True:
    model = Sequential()

    # model.add(Input(shape = (None, input_shape)))
    model.add(LSTM(32, return_sequences=True, input_shape=(None, step)))
    model.add(LSTM(32, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(32, activation="relu"))

    # output
    model.add(Dense(2, activation='softmax'))

    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
else:
    model = tf.keras.models.load_model("gender_classifier")

In [5]:
def prepare(s):
    s = s.replace("-", "")
    chars = string.ascii_lowercase + "'"
    try:
        a = []
        sequence = []
        last_i = 0
        for i, letter in zip(range(len(s)), s):
            last_i = i
            sequence.append(chars.index(letter) + 1)
            if len(sequence) == step:
                a.append(sequence)
                sequence = []
            if len(sequence) != 0 and len(sequence) < step and i == len(s) - 1:
                sequence.extend([0] * (step - len(sequence)))
                a.append(sequence)

        if len(a) < (maxlen / step):
            n = int(np.floor((maxlen / step))) - len(a)
            for _ in range(n):
                a.append([0] * step)
        return np.array(a)

    except Exception:
        import traceback
        traceback.print_exc()
        print("error processing word: {}".format(s))

In [6]:
with open("dataset/nomi_italiani_gendered.csv", "r") as csvfile:
    reader = csv.reader((l.replace("\0", "") for l in csvfile), delimiter=";", quotechar="|")
    nouns = []
    gender = []
    for row in reader:
        nouns.append(prepare(row[0]))
        gender.append([1, 0] if row[1] == "M" else [0, 1])
    p = np.random.permutation(len(nouns))
    nouns = np.array(nouns)[p]
    gender = np.array(gender)[p]

In [7]:
if TRAIN == True:
    model.fit(nouns[:-100], gender[:-100], epochs = 10, batch_size = 16)

In [8]:
model.evaluate(nouns[-100:], gender[-100:], batch_size = 16)

7/7 [==============================] - 1s 21ms/step - loss: 0.4628 - accuracy: 0.8200


[0.46277087926864624, 0.8199999928474426]

In [ ]:
while True:
        t = input("test noun - ctrl+c to close -> ")
        pred = model.predict(np.expand_dims(prepare(t), axis = 0))
        id = np.argmax(pred)
        print(["M", "F"][id], pred)